<a href="https://colab.research.google.com/github/jiwoong2/deeplearning/blob/main/%EC%9D%B8%EA%B3%B5%EC%8B%A0%EA%B2%BD%EB%A7%9D_%EA%B5%AC%ED%98%842.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys, os
import numpy as np
import pickle # 피클라이브러리는 텍스트파일의 비효율적인 저장방식을 효율적으로 개선
import tensorflow as tf

# 시그모이드, 소프트맥스 함수

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))  # np.exp(x)는 밑이 자연상수 e 이고 x가 지수인 수를 반환한다.

def softmax(x):
    if x.dim == 2:   # x가 행렬일 경우 scalar = 0, vector = 1, matrix = 2, 위의 경우 vector를 처리하므로 후에 배치처리때 사용됨.
        x = x.T
        x = x - np.max(x, axis = 0)
        y = np.exp(x) / np.sum(np.exp(x), axis = 0)
        return y.T

    x = x - np.max(x) # 오버플로를 방지하기위해 작성. 노트참고. , vector에 scalar를 빼면 numpy에서 알아서 원소별로 연산을 한다.
    return np.exp(x) / np.sum(np.exp(x))  # normalize

# Data 전처리

In [ ]:
Mnist = tf.keras.datasets.mnist
(x_train, t_train), (x_test, t_test) = Mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0 # normalize -> 많은 경우 피쳐가 여러개이므로 큰 숫자를 가지는 피쳐의 영향력이 너무 커지므로 모든 피쳐값을 0부터 1까지의 값으로 정규화 한다.
x_train = x_train.reshape(60000, 784) # flatten
x_test = x_test.reshape(10000, 784)

In [ ]:
# 아직 모델을 학습시키는 방법을 배우지 않았기 때문에 미리학습된 모델을 사용해 보는 과정.